In [1]:
import pandas as pd

honey_production = pd.read_csv('./honeyProduction/US_honey_production_dataset.csv')

In [2]:
honey_production.head()


,Unnamed: 0,state,colony_number,yield_per_colony,productions,stocks,average_price,value_of_prod,year
0,0,Alabama,9000,54,73000,73000,240.0,1166000,2010
1,1,Arizona,24000,77,665000,665000,152.0,2809000,2010
2,2,Arkansas,25000,60,360000,360000,147.0,2205000,2010
3,3,California,410000,67,6318000,6318000,155.0,42579000,2010
4,4,Colorado,34000,56,533000,533000,152.0,2894000,2010


In [3]:
honey_production.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        479 non-null    int64  
 1   state             479 non-null    object 
 2   colony_number     479 non-null    int64  
 3   yield_per_colony  479 non-null    int64  
 4   productions       479 non-null    int64  
 5   stocks            479 non-null    int64  
 6   average_price     479 non-null    float64
 7   value_of_prod     479 non-null    int64  
 8   year              479 non-null    int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 33.8+ KB


In [4]:
honey_production.shape

(479, 9)

In [5]:
year_counts = honey_production['year'].value_counts()
year_counts

2010    40
2011    40
2012    40
2014    40
2015    40
2016    40
2017    40
2018    40
2019    40
2020    40
2021    40
2013    39
Name: year, dtype: int64

In [6]:
uniqe_states = honey_production['state'].unique()
print(uniqe_states, len(uniqe_states))

['Alabama' 'Arizona' 'Arkansas' 'California' 'Colorado' 'Florida'
 'Georgia' 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas'
 'Kentucky' 'Louisiana' 'Maine' 'Michigan' 'Minnesota' 'Mississippi'
 'Missouri' 'Montana' 'Nebraska' 'NewJersey' 'NewMexico' 'NewYork'
 'NorthCarolina' 'NorthDakota' 'Ohio' 'Oregon' 'Pennsylvania'
 'SouthDakota' 'Tennessee' 'Texas' 'Utah' 'Vermont' 'Virginia'
 'Washington' 'WestVirginia' 'Wisconsin' 'Wyoming' 'SouthCarolina'] 41


In [7]:
df_temp = pd.read_csv('./states_avg_temp_formatted/Alabama.csv')

df_merged = pd.merge(honey_production, df_temp, on='year')

df_merged['state'] = 'Alabama'

df_merged = df_merged[['state', 'colony_number', 'yield_per_colony', 'productions', 'stocks', 'average_price', 'value_of_prod', 'year', 'avg_temp', 'anomaly']]

df_merged.to_csv('merged_data.csv', index=False)

In [8]:
import os
df_merged = pd.DataFrame()

for filename in os.listdir('./states_avg_temp_formatted'):
    if filename.endswith('.csv'):
        state = filename.split('.')[0].replace(' ', '')
        df_temp = pd.read_csv(f'./states_avg_temp_formatted/{filename}')
        df_temp_merged = pd.merge(honey_production[honey_production['state'] == state], df_temp, on='year')
        df_merged = pd.concat([df_merged, df_temp_merged], ignore_index=True)

df_merged.to_csv('merged_data.csv', index=False)

In [9]:
df_merged = pd.read_csv('merged_data.csv')
year_counts = df_merged['year'].value_counts()
year_counts

2010    39
2011    39
2012    39
2014    39
2015    39
2016    39
2017    39
2018    39
2019    39
2020    39
2021    39
2013    38
Name: year, dtype: int64

In [10]:
states_honey = honey_production['state'].unique().tolist()

states_merged = df_merged['state'].unique().tolist()

missing_states = set(states_honey) - set(states_merged)

print('Missing states:', list(missing_states))

Missing states: ['Hawaii']


In [11]:
df_merged2 = pd.DataFrame()

for filename in os.listdir('./states_avg_precipitation_formatted'):
    if filename.endswith('.csv'):
        state = filename.split('.')[0].replace(' ', '')
        df_precipitation = pd.read_csv(f'./states_avg_precipitation_formatted/{filename}')
        df_precipitation_merged = pd.merge(df_merged[df_merged['state'] == state], df_precipitation, on='year')
        df_merged2 = pd.concat([df_merged2, df_precipitation_merged], ignore_index=True)

df_merged2.to_csv('merged_data2.csv', index=False)

In [12]:

year_counts = df_merged2['year'].value_counts()
year_counts

2010    39
2011    39
2012    39
2014    39
2015    39
2016    39
2017    39
2018    39
2019    39
2020    39
2021    39
2013    38
Name: year, dtype: int64

In [13]:
missing_precipitation = df_merged2[df_merged2['precipitation'].isna()]
print('Number of rows with missing precipitation data:', missing_precipitation.shape[0])

Number of rows with missing precipitation data: 0


In [22]:
df_merged3 = df_merged2.drop(columns=['Unnamed: 0', "anomaly", "colony_number", "year"])

df_merged3.to_csv('merged_data3.csv', index=False)

In [23]:
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Apply the scaler to the numerical columns
numerical_cols = ["yield_per_colony", "productions", "stocks", "average_price", "value_of_prod", "avg_temp", "precipitation"]
df_merged3[numerical_cols] = scaler.fit_transform(df_merged3[numerical_cols])


In [24]:
df_merged3

,state,yield_per_colony,productions,stocks,average_price,value_of_prod,avg_temp,precipitation
0,Alabama,0.296703,0.004698,0.004698,0.273411,0.011098,0.697740,0.652376
1,Alabama,0.252747,0.003928,0.003928,0.286017,0.010667,0.728814,0.708895
2,Alabama,0.296703,0.004082,0.004082,0.281433,0.009914,0.759887,0.751766
3,Alabama,0.274725,0.003312,0.003312,0.324983,0.009555,0.694915,0.933526
4,Alabama,0.285714,0.001078,0.001078,0.402913,0.012820,0.677966,0.794316
...,...,...,...,...,...,...,...,...
462,Wyoming,0.285714,0.013402,0.013402,0.182874,0.036952,0.146893,0.194284
463,Wyoming,0.318681,0.012555,0.012555,0.000550,0.047034,0.124294,0.153500
464,Wyoming,0.318681,0.022645,0.022645,0.000000,0.034501,0.064972,0.189146
465,Wyoming,0.142857,0.045906,0.045906,0.000321,0.028235,0.135593,0.082852


In [25]:
state_dummies = pd.get_dummies(df_merged3['state'])

df_merged3 = pd.concat([df_merged3, state_dummies], axis=1)

df_merged3 = df_merged3.drop(columns=['state'])

In [27]:
df_merged3.to_csv('merged_data4.csv', index=False)


In [28]:
df_merged3 = df_merged3.drop(columns=['yield_per_colony'])


In [ ]:
#write to csv
df_merged3.to_csv('merged_data5.csv', index=False)